In [1]:
import pymysql
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import joblib
import os

def train_and_save_model():
    # 데이터베이스 연결 및 데이터 가져오기
    db = pymysql.connect(user='root', passwd='1004', host='localhost', db='cmsplusmain', charset='utf8')
    query = """
   SELECT 
    c.contract_id,
    m.member_name,
    YEAR(m.member_enroll_date) as enroll_year,
    DATEDIFF(c.contract_end_date, c.contract_start_date) as contract_duration,
    SUM(cp.contract_product_price * cp.contract_product_quantity) as total_contract_amount,
    p.payment_type,
    MAX(CASE WHEN c2.contract_id IS NOT NULL THEN 1 ELSE 0 END) as renewed
FROM 
    member m
JOIN 
    contract c ON m.member_id = c.member_id
JOIN 
    contract_product cp ON c.contract_id = cp.contract_id
LEFT JOIN 
    payment p ON c.payment_id = p.payment_id
LEFT JOIN 
    contract c2 ON m.member_id = c2.member_id AND c2.contract_start_date > c.contract_end_date
GROUP BY 
    m.member_id, m.member_name, YEAR(m.member_enroll_date), c.contract_id, c.contract_start_date, c.contract_end_date, p.payment_type
    """
    df = pd.read_sql(query, db)
    db.close()

    # 데이터 전처리
    df = pd.get_dummies(df, columns=['payment_type'])
    X = df.drop(['member_id', 'renewed'], axis=1)
    y = df['renewed']

    # 데이터 분할 및 스케일링
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # 모델 학습
    model = GradientBoostingClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    # 모델 평가
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    # 모델 및 관련 데이터 저장
    joblib.dump(model, 'gb_model.joblib')
    joblib.dump(scaler, 'scaler.joblib')
    joblib.dump(X.columns, 'feature_columns.joblib')
    
    print(f"Model trained and saved. Accuracy: {accuracy}")

def load_model_and_predict(new_data):
    # 저장된 모델 및 데이터 로드
    model = joblib.load('gb_model.joblib')
    scaler = joblib.load('scaler.joblib')
    feature_columns = joblib.load('feature_columns.joblib')

    # 새 데이터 전처리
    new_df = pd.DataFrame([new_data])
    new_df = pd.get_dummies(new_df, columns=['payment_type'])
    
    for col in feature_columns:
        if col not in new_df.columns:
            new_df[col] = 0
    
    new_df = new_df[feature_columns]
    new_data_scaled = scaler.transform(new_df)

    # 예측
    prediction_proba = model.predict_proba(new_data_scaled)
    renewal_probability = prediction_proba[0][1] * 100

    return {"new_member_renewal_probability": f"{renewal_probability:.2f}%"}

def execute_model(member_data):
    # 모델 파일이 없으면 학습 및 저장
    if not os.path.exists('gb_model.joblib'):
        train_and_save_model()
    
    # 저장된 모델로 예측
    result = load_model_and_predict(member_data)
    print(result)
    return result
